## Load Data

In [1]:
import pandas as pd

In [2]:
file = '../raw_data/data.csv'

df = pd.read_csv(file, low_memory = False)

## Data Inspection

In [3]:
df.head(3)

,complaint_id,date,time,precinct_number,offense_type,crime_completed,offense_level,borough,premise_desc,premise,...,suspect_age,suspect_race,suspect_sex,latitude,longitude,patrol_borough,metro,victim_age,victim_race,victim_sex
0,325341655,02/11/2015,15:00:00,73.0,OFFENSES AGAINST PUBLIC ADMINI,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE - PUBLIC HOUSING,...,25-44,BLACK,M,40.664239,-73.908425,PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M
1,393816841,03/17/2012,10:30:00,69.0,ASSAULT 3 & RELATED OFFENSES,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE-HOUSE,...,NaN,NaN,NaN,40.644590,-73.892672,PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,F
2,802896158,10/27/2016,13:48:00,71.0,HARRASSMENT 2,COMPLETED,VIOLATION,BROOKLYN,INSIDE,PUBLIC SCHOOL,...,<18,BLACK,M,40.658758,-73.942435,PATROL BORO BKLYN SOUTH,NaN,18-24,BLACK,M


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6983207 entries, 0 to 6983206
Data columns (total 22 columns):
 #   Column           Dtype  
---  ------           -----  
 0   complaint_id     int64  
 1   date             object 
 2   time             object 
 3   precinct_number  float64
 4   offense_type     object 
 5   crime_completed  object 
 6   offense_level    object 
 7   borough          object 
 8   premise_desc     object 
 9   premise          object 
 10  jurisdiction     object 
 11  park_name        object 
 12  suspect_age      object 
 13  suspect_race     object 
 14  suspect_sex      object 
 15  latitude         float64
 16  longitude        float64
 17  patrol_borough   object 
 18  metro            object 
 19  victim_age       object 
 20  victim_race      object 
 21  victim_sex       object 
dtypes: float64(3), int64(1), object(18)
memory usage: 1.1+ GB


## Data Cleaning: Datetime

### Time

In [5]:
def to_timestamp(column_name, df):
    '''converts given column to datetime.time dtype'''
    clean_df = df.copy()
    clean_df[column_name] = pd.to_datetime(clean_df[column_name], format = '%H:%M:%S').dt.time
    return clean_df

In [6]:
clean_df = to_timestamp('time', df)

In [7]:
type(clean_df['time'][3])

datetime.time

In order to access **only** the hour or minutes of the committed crime, use **.hour** or **.minute**, like so:

In [8]:
clean_df['hour'] = clean_df['time'].apply(lambda x: x.hour)

In [9]:
clean_df['hour'].head() # will convert to integer once NaN dropped

0    15.0
1    10.0
2    13.0
3    19.0
4    13.0
Name: hour, dtype: float64

In [10]:
clean_df['minute'] = clean_df['time'].apply(lambda x: x.minute)

In [11]:
clean_df['time'].isna().sum() # 48 NaN in 'time' column to be dropped

48

In [12]:
clean_df.head(3)

,complaint_id,date,time,precinct_number,offense_type,crime_completed,offense_level,borough,premise_desc,premise,...,suspect_sex,latitude,longitude,patrol_borough,metro,victim_age,victim_race,victim_sex,hour,minute
0,325341655,02/11/2015,15:00:00,73.0,OFFENSES AGAINST PUBLIC ADMINI,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE - PUBLIC HOUSING,...,M,40.664239,-73.908425,PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M,15.0,0.0
1,393816841,03/17/2012,10:30:00,69.0,ASSAULT 3 & RELATED OFFENSES,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE-HOUSE,...,NaN,40.644590,-73.892672,PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,F,10.0,30.0
2,802896158,10/27/2016,13:48:00,71.0,HARRASSMENT 2,COMPLETED,VIOLATION,BROOKLYN,INSIDE,PUBLIC SCHOOL,...,M,40.658758,-73.942435,PATROL BORO BKLYN SOUTH,NaN,18-24,BLACK,M,13.0,48.0


In [48]:
def to_timestamp(self):
    '''converts given column to datetime.time dtype'''
    df = self.data.copy()
    df['time'] = pd.to_datetime(df['time'], format = '%H:%M:%S').dt.time
    return df

### Date

In [13]:
df['date'].apply(type).value_counts() # mixed dtypes in 'date'

<class 'str'>      6982552
<class 'float'>        655
Name: date, dtype: int64

In [14]:
df[df['date'].apply(type) == float].head(3) # float dtypes are NaN

,complaint_id,date,time,precinct_number,offense_type,crime_completed,offense_level,borough,premise_desc,premise,...,suspect_age,suspect_race,suspect_sex,latitude,longitude,patrol_borough,metro,victim_age,victim_race,victim_sex
20568,389149830,NaN,13:55:00,43.0,DISORDERLY CONDUCT,COMPLETED,VIOLATION,BRONX,INSIDE,PUBLIC SCHOOL,...,<18,BLACK,F,40.822303,-73.856172,PATROL BORO BRONX,NaN,<18,BLACK,F
57141,550470474,NaN,18:00:00,49.0,SEX CRIMES,COMPLETED,MISDEMEANOR,BRONX,INSIDE,RESIDENCE - APT. HOUSE,...,NaN,UNKNOWN,F,40.856163,-73.844397,PATROL BORO BRONX,NaN,<18,WHITE HISPANIC,M
61657,901463880,NaN,10:00:00,67.0,GRAND LARCENY,COMPLETED,FELONY,BROOKLYN,INSIDE,RESIDENCE - APT. HOUSE,...,NaN,NaN,NaN,40.663057,-73.923641,PATROL BORO BKLYN SOUTH,NaN,25-44,BLACK,M


In [15]:
df['date'][0:3] # format of date is month / day / year

0    02/11/2015
1    03/17/2012
2    10/27/2016
Name: date, dtype: object

In [16]:
# drop NaN function (Camille)

def drop_miss(df):
    '''drop precinct and time missing values'''
    #drop precinct missing values
    not_unknown = df['precinct_number'] != -99.0
    not_nan = df['precinct_number'] == df['precinct_number']
    df = df[not_unknown & not_nan]
    #drop time and date missing values
    df = df[df['time'] == df['time']]
    df = df[df['date'] == df['date']]
    return df

In [17]:
# apply drop NaN
df = drop_miss(df)

In [18]:
from datetime import datetime

def to_iso_format(df, column):
    """Converts pd.series of strings to ISO datetime format"""
    series = df[column].apply(lambda x: \
                                  datetime.strptime(x, '%m/%d/%Y')) # converts to datetime, then ISO format
    return series

In [55]:
from datetime import datetime

def to_date_format(self):
    """Converts column 'date' from string to datetime and returns filtered df (complaints from 2007 onwards)"""
    df = self.data.copy()
    df['date'] = df['date'].apply(lambda x: \
                                  datetime.strptime(x, '%m/%d/%Y')) # converts to datetime, then ISO format
    df = df[df['date'] > datetime(2006, 12, 31, 0, 0)]
    return df

In [19]:
def to_date_format_test(s):
    """Convert a single US date to ISO format"""
    return datetime.strptime(s, '%m/%d/%Y')

# Create a new feature iso_date 
df["iso_date"] = df["date"].apply(to_date_format)

In [20]:
type(df['iso_date'][0])

datetime.datetime

In [21]:
df['iso_date'][0]

datetime.datetime(2015, 2, 11, 0, 0)

In [22]:
test_df = df.copy()

In [23]:
type(test_df['date'][0])

str

In [24]:
test_df['date'][0]

'02/11/2015'

In [25]:
test_df['date'] = to_iso_format(df, 'date')

In [26]:
test_df['date'][0]

datetime.datetime(2015, 2, 11, 0, 0)

In [27]:
type(test_df['date'][0])

datetime.datetime

In [28]:
test_df['date'].apply(type).value_counts()

<class 'datetime.datetime'>    6979589
Name: date, dtype: int64

In [53]:
# Filter df
modern_df = test_df.copy()
modern_df = modern_df[test_df['date'] > datetime(2006, 12, 31, 0, 0)]

In [51]:
test_df['date'][0] > datetime(2006, 12, 31, 0, 0)

True

In [52]:
datetime(1015, 2, 14, 0, 0) > datetime(2006, 12, 31, 0, 0)

False

In [ ]:
np.sort(modern_df['date'].unique())

In [54]:
import numpy as np

np.sort(modern_df['date'].unique())

array([datetime.datetime(2007, 1, 1, 0, 0),
       datetime.datetime(2007, 1, 2, 0, 0),
       datetime.datetime(2007, 1, 3, 0, 0), ...,
       datetime.datetime(2019, 12, 29, 0, 0),
       datetime.datetime(2019, 12, 30, 0, 0),
       datetime.datetime(2019, 12, 31, 0, 0)], dtype=object)

In [31]:
other_df = test_df.copy()
other_df[test_df["iso_date"] < datetime(2007, 1, 1, 0, 0)].shape

(548288, 23)

In [32]:
df['date'].str.contains('1018').sum() # date with wrong year values to be filtered out

31

In [33]:
error_df = test_df.copy()
error_df[test_df["iso_date"] < datetime(1900, 1, 1, 0, 0)].shape # 155 rows with wrong vbalues

(155, 23)

## Data Viz: Time

### Generate New Columns

In [35]:
viz_df = df.copy()

### Group by Year

### Group by Month

### Group by Week

### Group by DOW

### Group by Hour